<a href="https://colab.research.google.com/github/Byeongwoo-An/llm-reasoning/blob/main/SPCA_Embedding4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import gc
import random
import os

In [ ]:
def data_preprocessing(data, level):
  X = np.array([item['embeds'] for item in data])
  y = np.array([int(item['is_correct']) for item in data])
  steps = np.array([item['step_num'] for item in data])
  final = np.array([item['is_completed'] for item in data])

  unique_steps = np.unique(steps)

  class_ratio_results = []

  total_count = len(y)
  pos_ratio_all = y.mean()
  neg_ratio_all = 1 - pos_ratio_all

  class_ratio_results.append({
      'step_num': 'all',
      'count': total_count,
      'positive_ratio': round(pos_ratio_all, 4),
      'negative_ratio': round(neg_ratio_all, 4)
  })

  # step별 결과
  for step in unique_steps:
      mask = (steps == step)
      total = mask.sum()
      pos_ratio = y[mask].mean()
      neg_ratio = 1 - pos_ratio
      class_ratio_results.append({
          'step_num': step,
          'count': total,
          'positive_ratio': round(pos_ratio, 4),
          'negative_ratio': round(neg_ratio, 4)
      })

  class_ratio = pd.DataFrame(class_ratio_results)
  class_ratio.to_csv(f"class_ratio_by_step_{level}.csv", index=False)

  final_mask = (final == True)
  steps_final = steps[final_mask]
  y_final = y[final_mask]

  final_class_ratio_result = []
  for step in np.unique(steps_final):
      idx = (steps_final == step)
      count = idx.sum()
      pos_ratio = y_final[idx].mean()
      neg_ratio = 1 - pos_ratio
      final_class_ratio_result.append({
          'step_num': int(step),
          'final_step_count': count,
          'positive_ratio': round(pos_ratio, 4),
          'negative_ratio': round(neg_ratio, 4)
      })

  df_termination_step = pd.DataFrame(final_class_ratio_result)
  df_termination_step.to_csv(f"class_ratio_by_termination_step_{level}.csv", index=False)

  X_train, X_test, y_train, y_test, step_train, step_test = train_test_split(X, y, steps, test_size=0.3, random_state=42)

  return X_train, X_test, y_train, y_test, step_train, step_test

In [ ]:
def run_logistic_regression(X_train, y_train, X_test, y_test, step_test, level):

    C_values = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    results = []

    for C in C_values:
        clf = LogisticRegression(penalty='l2', C=C, max_iter=1000)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        error_rate = 1 - accuracy_score(y_test, y_pred)

        print(f"C={C:>7} | Error Rate: {error_rate:.4f}, n_iter={clf.n_iter_[0]}")

        results.append({
            'C': C,
            'model': clf,
            'y_pred': y_pred,
            'error_rate': error_rate
        })

    best_result = min(results, key=lambda x: x['error_rate'])
    best_C = best_result['C']
    y_pred = best_result['y_pred']

    print(f"\nBest C = {best_C} with error rate = {best_result['error_rate']:.4f}")

    logreg_results = []

    acc_total = accuracy_score(y_test, y_pred)
    err_total = 1 - acc_total

    logreg_results.append({
        'step_num': 'Total',
        'count': len(y_test),
        'error_rate': round(err_total, 4)
    })

    for step in np.unique(step_test):
        idx = (step_test == step)
        if np.sum(idx) == 0:
            continue
        acc_step = accuracy_score(y_test[idx], y_pred[idx])
        err_step = 1 - acc_step
        logreg_results.append({
            'step_num': int(step),
            'count': int(np.sum(idx)),
            'error_rate': round(err_step, 4)
        })

    df_logreg = pd.DataFrame(logreg_results)
    df_logreg.to_csv(f"logistic_result_{level}.csv", index=False)

In [ ]:
class NN(nn.Module):
    def __init__(self, input_dim, dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, dim)  # embedding layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(dim, 1)          # classifier layer

    def forward(self, x):
        x = self.fc1(x)
        x_embed = self.relu(x)  # low-dimensional embedding
        x = self.fc2(x_embed)
        return x, x_embed

def run_NN(X, y, level, dim_list, lr_list, num_epochs_list):

  seed = 42
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  g = torch.Generator()
  g.manual_seed(seed)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
  y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(device)

  dataset = TensorDataset(X_tensor, y_tensor)
  loader = DataLoader(dataset, batch_size=512, shuffle=True, generator=g)

  for num_epochs in num_epochs_list:
    for lr in lr_list:
        nn_results = []

        for dim in tqdm(dim_list, desc=f"LR={lr}"):

          model = NN(input_dim=X_tensor.shape[1], dim=dim).to(device)
          criterion = nn.BCEWithLogitsLoss()
          optimizer = optim.Adam(model.parameters(), lr=lr)

          # train the model
          model.train()
          for epoch in tqdm(range(num_epochs), desc=f"epoch", leave=False):
              for xb, yb in loader:
                  optimizer.zero_grad()
                  logits, _ = model(xb)
                  loss = criterion(logits, yb)
                  loss.backward()
                  optimizer.step()

          # model save
          model.eval()
          model_filename = f"nn_model_{level}_dim-{dim}_lr-{str(lr).replace('.', '')}_epoch-{num_epochs}.pth"
          torch.save(model.state_dict(), model_filename)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/MyDrive/Tuan/embedding_data/"
file_list = [f for f in os.listdir(folder_path) if f.endswith(".pkl")]
file_list.sort()

In [ ]:
dim_list = [16, 32, 64, 128, 256]
learning_rate_list = [0.00025]
num_epochs_list=[25]

for filename in file_list:
    parts = filename.split("--")
    level = next((p for p in parts if p.startswith("level-")), "unknown")

    file_path = os.path.join(folder_path, filename)
    with open(file_path, "rb") as f:
      data = pickle.load(f)
      X = np.array([item['embeds'] for item in data])
      y = np.array([int(item['is_correct']) for item in data])
      run_NN(X, y, level, dim_list, learning_rate_list, num_epochs_list)

LR=0.00025: 100%|██████████| 5/5 [11:55<00:00, 143.16s/it]


In [ ]:
model = NN(input_dim=X.shape[1], dim=64)
model.load_state_dict(torch.load("nn_model_level-level-4_dim-64_lr-000025_epoch-25.pth"))

<All keys matched successfully>